In [ ]:
# import liabraries
import pandas as pd
import json

In [ ]:
df = pd.read_csv('cs_shorts4.csv',parse_dates=['fetching_date','video_published_at'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848 entries, 0 to 847
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   video_id              848 non-null    object             
 1   fetching_date         848 non-null    datetime64[ns]     
 2   video_title           848 non-null    object             
 3   video_published_at    848 non-null    datetime64[ns, UTC]
 4   video_description     628 non-null    object             
 5   channel_id            848 non-null    object             
 6   channel_title         848 non-null    object             
 7   tags                  848 non-null    object             
 8   category_id           848 non-null    int64              
 9   default_language      848 non-null    object             
 10  duration              848 non-null    object             
 11  dimension             848 non-null    object             
 12  definiti

In [ ]:
df.sort_values(by='video_view_count', ascending=False).head(400)[['video_like_count','video_comment_count']]

,video_like_count,video_comment_count
639,707761.0,691.0
283,1680604.0,3589.0
208,954373.0,5184.0
836,445465.0,581.0
47,1251047.0,809423.0
...,...,...
644,1878.0,2.0
84,4591.0,36.0
684,932.0,144.0
409,2129.0,32.0


In [ ]:
df['fetching_date_UTC'] = df['fetching_date'].dt.tz_localize('UTC')

In [ ]:
df['views_per_hour'] = df['video_view_count']/(df['fetching_date_UTC'] - df['video_published_at']).dt.total_seconds()*3600
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848 entries, 0 to 847
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   video_id              848 non-null    object             
 1   fetching_date         848 non-null    datetime64[ns]     
 2   video_title           848 non-null    object             
 3   video_published_at    848 non-null    datetime64[ns, UTC]
 4   video_description     628 non-null    object             
 5   channel_id            848 non-null    object             
 6   channel_title         848 non-null    object             
 7   tags                  848 non-null    object             
 8   category_id           848 non-null    int64              
 9   default_language      848 non-null    object             
 10  duration              848 non-null    object             
 11  dimension             848 non-null    object             
 12  definiti

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['video_view_count_scaled','video_like_count_scaled','video_comment_count_scaled','views_per_hour_scaled']] = scaler.fit_transform(df[['video_view_count','video_like_count','video_comment_count','views_per_hour']])

In [ ]:

threshold = df['video_view_count'].quantile(0.7)
df['performance_label'] = np.where(df['video_view_count'] >= threshold, 'High', 'Low')


In [ ]:
df = df[df['views_per_hour'] > 0]

In [ ]:
metrics = ['video_view_count_scaled', 'video_like_count_scaled',
           'video_comment_count_scaled', 'views_per_hour_scaled']

json_data = []

for _, row in df.iterrows():
    prompt = (
        f"Total scaled views: {row['video_view_count_scaled']}, "
        f"Total scaled likes: {row['video_like_count_scaled']}, "
        f"Total scaled comments: {row['video_comment_count_scaled']}, "
        f"Scaled views per hour: {row['views_per_hour_scaled']}, "
        f"Performance: {row['performance_label']}."
    )

    completion = (
        f"Title: {row['video_title']}, \n"
        f"Description: {row['video_description']}, \n"
        f"Tags: {row['tags']}"
    )

    json_data.append({'prompt': prompt, 'completion': completion})

# Write to JSON lines file (one JSON object per line)
with open('youtube_cs_shorts_json.json', 'w', encoding='utf-8') as json_file:
    for item in json_data:
        json_file.write(json.dumps(item, ensure_ascii=False) + '\n')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "HuggingFaceTB/SmolLM3-3B"
device = "cuda"  # for GPU usage or "cpu" for CPU usage

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device)

from peft  import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
peft_model = get_peft_model(model, lora_config)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/182 [00:00<?, ?B/s]

In [ ]:

from datasets import load_dataset
dataset = load_dataset('json', data_files='youtube_cs_shorts_json.json')

def tokenize_function(examples):
    return tokenizer(
        examples['prompt'],
        text_target=examples['completion'],
        padding='max_length',
        truncation=True,
        max_length=256
    )
tokenized_datasets = dataset.map(tokenize_function, batched=True)



Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/846 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./smollm3-cs-shorts-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=4,
    learning_rate=2e-4,
    logging_steps=10,
    save_total_limit=3,
    fp16=True,
    save_strategy="steps",
    save_steps=10,
    lr_scheduler_type="cosine",
    report_to=[]
)
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets['train']

)

trainer.train()


Step,Training Loss
10,12.260800
20,5.592200
30,4.907400
40,5.374500
50,4.486100
60,5.108100
70,4.898100
80,4.674700
90,4.871400
100,4.669300


TrainOutput(global_step=212, training_loss=5.2081683716684015, metrics={'train_runtime': 762.5103, 'train_samples_per_second': 4.438, 'train_steps_per_second': 0.278, 'total_flos': 1.4638445607518208e+16, 'train_loss': 5.2081683716684015, 'epoch': 4.0})

In [ ]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./smollm3-cs-shorts-finetuned2",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=10,
    save_total_limit=3,
    fp16=True,
    save_strategy="steps",
    save_steps=10,
    lr_scheduler_type="cosine",
    report_to=[]
)
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets['train']

)

trainer.train()


Step,Training Loss
10,5.102300
20,5.001100
30,4.390500
40,4.990700
50,4.730300
60,4.242100
70,5.269200
80,4.814600
90,4.572800
100,3.871400


TrainOutput(global_step=106, training_loss=4.732408253651745, metrics={'train_runtime': 193.2432, 'train_samples_per_second': 4.378, 'train_steps_per_second': 0.549, 'total_flos': 3659611401879552.0, 'train_loss': 4.732408253651745, 'epoch': 1.0})

In [ ]:
def generate_text(prompt, max_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output_sequences = peft_model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.6,
        top_p=0.95,
        do_sample=True
    )
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    return generated_text

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "HuggingFaceTB/SmolLM3-3B"
device = "cuda"  # for GPU usage or "cpu" for CPU usage

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model2 = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device)

from peft  import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
peft_model = get_peft_model(model2, lora_config)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

from datasets import load_dataset
dataset = load_dataset('json', data_files='youtube_cs_shorts_json.json')

def tokenize_function(examples):
    return tokenizer(
        examples['prompt'],
        text_target=examples['completion'],
        padding='max_length',
        truncation=True,
        max_length=256
    )
tokenized_datasets = dataset.map(tokenize_function, batched=True)



In [ ]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./smollm3-cs-shorts-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=1e-4,
    logging_steps=10,
    save_total_limit=3,
    fp16=True,
    save_strategy="steps",
    save_steps=10,
    lr_scheduler_type="cosine",
    report_to=[]
)
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets['train']

)

trainer.train()


Step,Training Loss
10,17.138800
20,8.066500
30,5.927600
40,6.041600
50,5.319400
60,4.772500
70,5.780300
80,5.305400
90,5.031100
100,4.275000


TrainOutput(global_step=106, training_loss=6.713857614769126, metrics={'train_runtime': 208.1594, 'train_samples_per_second': 4.064, 'train_steps_per_second': 0.509, 'total_flos': 3657120438288384.0, 'train_loss': 6.713857614769126, 'epoch': 1.0})

In [ ]:
def generate_text(prompt, max_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output_sequences = peft_model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.6,
        top_p=0.95,
        do_sample=True
    )
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    return generated_text

In [ ]:

print(generate_text("Total scaled views: 5000, Total scaled likes: 400, Total scaled comments: 50, Scaled views per hour: 100. Performance: high."))

Total scaled views: 5000, Total scaled likes: 400, Total scaled comments: 50, Scaled views per hour: 100. Performance: high.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

testttttt


In [ ]:
import gradio as gr

demo = gr.Interface(fn=greeting, inputs="text", outputs="text")
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
